### DN1 - Naloga 1
Avtor: Urh Primožič

Najprej razdelimo podatke na testno in učno množico.

In [10]:
import pandas as pd #cvs importing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

RANDOM_SEED = 420
df = pd.read_csv('podatki_DN1_1.csv', header=0)
columns = df.columns[1:] #1. stolpec je le dekoracijski
df = df[columns]
X = df[['x1', 'x2', 'x3', 'x4', 'x5']]
y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=2/3, random_state=RANDOM_SEED)

Model najbližjih sosedov.

In [11]:
knn3 = KNeighborsRegressor(n_neighbors=3)
knn3.fit(X_train, y_train)
mse3 = mean_squared_error(y_test, knn3.predict(X_test))
print('Napaka knn modela za k=3 je ', mse3)

Napaka knn modela za k=3 je  0.012423075075045954


Iskanje optimalnega števila sosedov

In [12]:
from sklearn.utils import resample
k_range = list(range(1,100))
def boot(X, y,model, n_iterations):
    '''
    Bootstraping.
    Parameters
    ------------
    model - class with mthods fit() and predict()

    Returns
    -------
    Tabelo mse za vsako iteracijo
    '''
    vzorci = [resample(X, y) for i in range(n_iterations)]
    mse = []
    for i in range(n_iterations):
        # naučiš se na enem vzorcu
        X_train = vzorci[i][0]
        y_train = vzorci[i][1]
        model.fit(X_train, y_train)
        # potestiraš na ostalih
        # i-ta napaka je mse na ostalih vzorcih
        # ker so vsi vzorci enako veliki (veliki so |X|)
        # dobimo enako, če seštejemo vse napake na vzorcih in delimo s št_ ostalih vzorccev
        err = 0
        for j in range(n_iterations):
            if j == i:
                continue
            X_test = vzorci[j][0]
            y_test = vzorci[j][1]
            err += mean_squared_error(y_test, model.predict(X_test))
        err /= n_iterations -1
        mse.append(err)
    return  mse


def knn_boot(X, y, n_iterations, param_space):
    '''
    Poišče najbolši k za knn z bootstrapingom

    Returns
    ---------
    dictionary with keys
        'model'
        'mse'
    '''
    napake = []
    for k in param_space:
        knn = KNeighborsRegressor(n_neighbors=k)



ValueError: Expected cv as an integer, cross-validation object (from sklearn.model_selection) or an iterable. Got <function resample at 0x7f1a9a5f2ca0>.